# SciPy: Signal Processing and Optimization

Demonstrates signal processing, FFT, digital filtering, and numerical optimization with SciPy.

**Modules:**
- [`scipy.fft`](https://docs.scipy.org/doc/scipy/tutorial/fft.html) — Fast Fourier Transform and spectral analysis
- [`scipy.signal`](https://docs.scipy.org/doc/scipy/tutorial/signal.html) — Digital filters, spectrograms, peak detection
- [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html) — Minimization, curve fitting, root finding
- [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) — Hypothesis tests, distributions, correlation
- [`scipy.interpolate`](https://docs.scipy.org/doc/scipy/reference/interpolate.html) — Spline and RBF interpolation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, fft, optimize, stats, interpolate

%matplotlib inline
rng = np.random.default_rng(seed=0)

## Signal Generation

Create a composite signal with two sinusoidal components buried in Gaussian noise.

In [ ]:
fs = 1000      # Hz
T = 2.0        # seconds
t = np.linspace(0, T, int(fs * T), endpoint=False)

freq1, freq2 = 50.0, 120.0
clean = np.sin(2 * np.pi * freq1 * t) + 0.5 * np.sin(2 * np.pi * freq2 * t)
noise_std = 0.4
noisy = clean + rng.normal(scale=noise_std, size=len(t))

snr_db = 10 * np.log10((clean**2).mean() / noise_std**2)
print(f"fs={fs} Hz  |  T={T} s  |  {len(t)} samples")
print(f"Signal: {freq1} Hz + {freq2} Hz  |  SNR: {snr_db:.1f} dB")

## FFT Analysis

The FFT decomposes the time-domain signal into its frequency components.

In [ ]:
N = len(t)
freqs = fft.rfftfreq(N, d=1/fs)
spectrum_noisy = np.abs(fft.rfft(noisy)) * 2 / N
spectrum_clean = np.abs(fft.rfft(clean)) * 2 / N

peaks_idx, _ = signal.find_peaks(spectrum_noisy, height=0.1, distance=10)
peak_freqs = freqs[peaks_idx]

print(f"Dominant frequencies detected: {peak_freqs[:3].tolist()} Hz")

fig, axes = plt.subplots(2, 1, figsize=(12, 7))

axes[0].plot(t[:500], clean[:500], label='Clean', linewidth=1.2, color='steelblue')
axes[0].plot(t[:500], noisy[:500], label='Noisy', linewidth=0.6, alpha=0.7, color='tomato')
axes[0].set_title("Time Domain")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

axes[1].semilogy(freqs, spectrum_noisy, label='Noisy', linewidth=0.8, alpha=0.7, color='tomato')
axes[1].semilogy(freqs, spectrum_clean, label='Clean', linewidth=1.5, color='steelblue')
axes[1].plot(freqs[peaks_idx], spectrum_noisy[peaks_idx], 'x', color='black', markersize=10, label='Peaks')
axes[1].set_title("Frequency Domain (FFT)")
axes[1].set_xlabel("Frequency (Hz)")
axes[1].set_ylabel("Amplitude")
axes[1].set_xlim([0, 300])
axes[1].legend()

plt.tight_layout()
plt.show()

## Digital Filtering

Butterworth filters with `filtfilt` (zero-phase, no time delay) and an IIR notch filter.

In [ ]:
def apply_butter(data, cutoff, fs, btype, order=5):
    """Zero-phase Butterworth filter."""
    nyq = fs / 2
    norm = np.array(cutoff) / nyq
    b, z = signal.butter(order, norm, btype=btype)
    return signal.filtfilt(b, z, data)

b_notch, a_notch = signal.iirnotch(w0=50, Q=30, fs=fs)

filters = {
    'Lowpass < 80 Hz':   apply_butter(noisy, 80, fs, 'low'),
    'Highpass > 80 Hz':  apply_butter(noisy, 80, fs, 'high'),
    'Bandpass 40–70 Hz': apply_butter(noisy, [40, 70], fs, 'band'),
    'Notch at 50 Hz':    signal.filtfilt(b_notch, a_notch, noisy),
}

fig, axes = plt.subplots(2, 2, figsize=(13, 8))
colors = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6']

for ax, (fname, filt), color in zip(axes.ravel(), filters.items(), colors):
    ax.plot(t[:500], noisy[:500], alpha=0.35, linewidth=0.6, color='gray', label='Noisy')
    ax.plot(t[:500], filt[:500], linewidth=1.5, color=color, label=fname)
    ax.set_title(fname)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Amplitude")
    ax.legend(fontsize=8)

plt.suptitle("Digital Filtering Results", fontsize=13)
plt.tight_layout()
plt.show()

## Spectrogram (STFT)

A chirp sweeps from 50 to 300 Hz — the spectrogram reveals how frequency changes over time.

In [ ]:
chirp_signal = signal.chirp(t, f0=50, f1=300, t1=T, method='linear')
chirp_noisy = chirp_signal + rng.normal(scale=0.3, size=len(t))

stft = fft.ShortTimeFFT(win=signal.windows.hann(256), hop=64, fs=fs, mfft=512)
Sx = stft.spectrogram(chirp_noisy)

fig, axes = plt.subplots(2, 1, figsize=(12, 7))
axes[0].plot(t, chirp_signal, linewidth=0.8, color='steelblue')
axes[0].set_title("Chirp Signal (50 → 300 Hz sweep)")
axes[0].set_ylabel("Amplitude")

im = axes[1].imshow(
    10 * np.log10(Sx + 1e-10), aspect='auto', origin='lower',
    extent=stft.extent(len(chirp_noisy)), cmap='inferno'
)
axes[1].set_ylim([0, 400])
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Frequency (Hz)")
axes[1].set_title("Spectrogram (ShortTimeFFT)")
plt.colorbar(im, ax=axes[1], label="Power (dB)")

plt.tight_layout()
plt.show()

## Optimization

SciPy provides local and global minimizers, curve fitting, and root finding.

In [ ]:
# Rosenbrock banana function — classic optimization test
def rosenbrock(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

res_nm = optimize.minimize(rosenbrock, [-1.0, 2.0], method='Nelder-Mead')
res_bfgs = optimize.minimize(rosenbrock, [-1.0, 2.0], method='BFGS', jac='2-point')

print("Rosenbrock minimization (global minimum at [1, 1], f=0):")
print(f"  Nelder-Mead : x={res_nm.x.round(4)}, f={res_nm.fun:.2e}, iters={res_nm.nit}")
print(f"  BFGS        : x={res_bfgs.x.round(4)}, f={res_bfgs.fun:.2e}, iters={res_bfgs.nit}")

In [ ]:
# Curve fitting: damped oscillation
def damped_oscillation(t, amp, decay, freq, phi):
    return amp * np.exp(-decay * t) * np.cos(2 * np.pi * freq * t + phi)

true_params = (2.5, 0.8, 5.0, np.pi/4)
t_fit = np.linspace(0, 3, 300)
y_data = damped_oscillation(t_fit, *true_params) + rng.normal(scale=0.15, size=300)

popt, pcov = optimize.curve_fit(damped_oscillation, t_fit, y_data, p0=[2.0, 1.0, 4.5, 0.0], maxfev=5000)
perr = np.sqrt(np.diag(pcov))

print("Damped oscillation curve fit:")
for name, true, fit, err in zip(['amplitude', 'decay', 'frequency', 'phase'], true_params, popt, perr):
    print(f"  {name:<12}: true={true:.3f}  fitted={fit:.3f} ± {err:.3f}")

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

x_g, y_g = np.meshgrid(np.linspace(-2, 2, 200), np.linspace(-1, 3, 200))
Z = (1-x_g)**2 + 100*(y_g-x_g**2)**2
axes[0].contourf(x_g, y_g, np.log1p(Z), levels=40, cmap='viridis')
axes[0].plot(*res_nm.x, 'rs', markersize=10, label='Nelder-Mead')
axes[0].plot(*res_bfgs.x, 'w^', markersize=10, label='BFGS')
axes[0].set_title("Rosenbrock Function")
axes[0].legend(fontsize=8)

axes[1].scatter(t_fit, y_data, s=4, alpha=0.5, label='Data', color='steelblue')
axes[1].plot(t_fit, damped_oscillation(t_fit, *true_params), '--', linewidth=2, label='True', color='green')
axes[1].plot(t_fit, damped_oscillation(t_fit, *popt), linewidth=2, label='Fitted', color='red')
axes[1].set_title("Curve Fitting: Damped Oscillation")
axes[1].set_xlabel("Time (s)")
axes[1].legend()

plt.tight_layout()
plt.show()

## Statistical Analysis

Hypothesis testing, effect size, and distribution fitting.

In [ ]:
group_a = rng.normal(loc=100, scale=15, size=80)
group_b = rng.normal(loc=108, scale=18, size=75)

_, p_a = stats.shapiro(group_a)
_, p_b = stats.shapiro(group_b)
_, levene_p = stats.levene(group_a, group_b)
t_stat, t_p = stats.ttest_ind(group_a, group_b, equal_var=(levene_p > 0.05))

pooled_std = np.sqrt(((len(group_a)-1)*group_a.std()**2 + (len(group_b)-1)*group_b.std()**2) /
                     (len(group_a)+len(group_b)-2))
cohens_d = (group_b.mean() - group_a.mean()) / pooled_std

print(f"Group A: mean={group_a.mean():.2f}, std={group_a.std():.2f}, n={len(group_a)}")
print(f"Group B: mean={group_b.mean():.2f}, std={group_b.std():.2f}, n={len(group_b)}")
print(f"\nShapiro-Wilk  : A p={p_a:.3f}, B p={p_b:.3f}")
print(f"Levene        : p={levene_p:.3f}")
print(f"t-test        : t={t_stat:.4f}, p={t_p:.4f} → {'Significant' if t_p < 0.05 else 'Not significant'}")
print(f"Cohen's d     : {cohens_d:.3f} ({'small' if abs(cohens_d)<0.5 else 'medium' if abs(cohens_d)<0.8 else 'large'})")

In [ ]:
x_range = np.linspace(min(group_a.min(), group_b.min())-10, max(group_a.max(), group_b.max())+10, 300)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].hist(group_a, bins=20, alpha=0.6, density=True, label='Group A', color='steelblue')
axes[0].hist(group_b, bins=20, alpha=0.6, density=True, label='Group B', color='tomato')
axes[0].plot(x_range, stats.norm.pdf(x_range, group_a.mean(), group_a.std()), '--', color='steelblue')
axes[0].plot(x_range, stats.norm.pdf(x_range, group_b.mean(), group_b.std()), '--', color='tomato')
axes[0].set_title(f"Group Comparison (p={t_p:.3f})")
axes[0].legend()

(osm, osr), (slope, intercept, r) = stats.probplot(group_a, dist='norm')
axes[1].scatter(osm, osr, s=15, color='steelblue', alpha=0.7)
axes[1].plot(osm, slope*np.array(osm)+intercept, 'r--', linewidth=2)
axes[1].set_title(f"Q-Q Plot (Group A), r={r:.3f}")
axes[1].set_xlabel("Theoretical Quantiles")
axes[1].set_ylabel("Sample Quantiles")

x_corr = rng.normal(size=100)
y_corr = 0.7 * x_corr + rng.normal(scale=0.5, size=100)
pearson_r, pearson_p = stats.pearsonr(x_corr, y_corr)
m, b = np.polyfit(x_corr, y_corr, 1)
axes[2].scatter(x_corr, y_corr, s=20, alpha=0.6, color='steelblue')
axes[2].plot(x_corr, m*x_corr+b, 'r--', linewidth=2)
axes[2].set_title(f"Pearson r={pearson_r:.3f}  (true ρ≈0.7)")
axes[2].set_xlabel("x")
axes[2].set_ylabel("y")

plt.tight_layout()
plt.show()

## Interpolation

Compare linear, cubic spline, and RBF interpolation on sparse data.

In [ ]:
x_sparse = np.array([0, 1, 2, 3, 5, 7, 8, 9, 10.0])
y_sparse = np.sin(x_sparse) + rng.normal(scale=0.05, size=len(x_sparse))
x_dense = np.linspace(0, 10, 500)

linear_f = interpolate.interp1d(x_sparse, y_sparse, kind='linear')
cubic_f = interpolate.CubicSpline(x_sparse, y_sparse)
rbf_f = interpolate.RBFInterpolator(x_sparse[:, None], y_sparse, kernel='thin_plate_spline')

fig, ax = plt.subplots(figsize=(11, 5))
ax.plot(x_dense, np.sin(x_dense), 'k-', linewidth=1.5, label='True sin(x)', alpha=0.4)
ax.scatter(x_sparse, y_sparse, s=60, zorder=5, color='black', label='Sparse data')
ax.plot(x_dense, linear_f(x_dense), '--', linewidth=1.5, label='Linear', color='tomato')
ax.plot(x_dense, cubic_f(x_dense), '-', linewidth=1.5, label='Cubic Spline', color='steelblue')
ax.plot(x_dense, rbf_f(x_dense[:, None]), '-', linewidth=1.5, label='RBF (thin plate)', color='seagreen')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Interpolation Methods Comparison")
ax.legend()
plt.tight_layout()
plt.show()

for name, yi in [('Linear', linear_f(x_dense)), ('Cubic Spline', cubic_f(x_dense)),
                  ('RBF', rbf_f(x_dense[:, None]))]:
    rmse = np.sqrt(np.mean((yi - np.sin(x_dense))**2))
    print(f"  {name:<22}: RMSE = {rmse:.5f}")

---
## Summary

1. **FFT** — Decomposes signals into frequency components; `rfft` for real signals
2. **Filtering** — Butterworth filters for lowpass/highpass/bandpass; IIR notch for specific frequencies; `filtfilt` for zero-phase
3. **Spectrogram** — STFT reveals frequency evolution over time; essential for non-stationary signals
4. **Optimization** — `minimize` supports many methods; `curve_fit` for parameter estimation
5. **Statistics** — Shapiro-Wilk, t-test, Levene's test; Cohen's d for effect size
6. **Interpolation** — Cubic splines and RBF outperform linear for smooth underlying functions

**Resources:**
- [SciPy Signal Processing Tutorial](https://docs.scipy.org/doc/scipy/tutorial/signal.html)
- [SciPy Optimization](https://docs.scipy.org/doc/scipy/reference/optimize.html)
- [SciPy Stats](https://docs.scipy.org/doc/scipy/reference/stats.html)